In [1]:
# ==============================================================================
# Imports
# ==============================================================================
import networkx as nx
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
from typing import List, Dict, Set, Tuple, Any, Optional
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack as sparse_hstack
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import confusion_matrix
import time
import os
import warnings

# ==============================================================================
# Agent Definition (Deterministic Preference + Explore Goal - Agent V5)
# ==============================================================================
# (Class Agent identical to previous version)
class Agent:
    """ Agent with Preference-Driven Goal: Find N preferred nodes using
        Deterministic Preference strategy (always choose preferred if available). """
    def __init__(self, start_node: int, color_preference: str,
                 target_preferred_count: int, max_steps: int):
        self.start_node: int = start_node; self.color_preference: str = color_preference
        self.target_preferred_count: int = target_preferred_count; self.max_steps: int = max_steps
        self.current_node: int = start_node; self.visited_nodes: Set[int] = {start_node}
        self.path: List[int] = [start_node]; self.preferred_nodes_visited: Set[int] = set()
        self.visited_shapes: List[str] = []; self.visited_shininess: List[str] = []

    def find_next_node(self, graph: nx.Graph, rng: random.Random) -> Optional[int]:
        """ Determines next node using Deterministic Preference strategy (V5 logic). """
        neighbors = list(graph.neighbors(self.current_node)); unvisited_neighbors = [n for n in neighbors if n not in self.visited_nodes]
        if not unvisited_neighbors: return None
        # Use safe graph node access
        preferred_unvisited_neighbors = [n for n in unvisited_neighbors if graph.nodes.get(n, {}).get('color') == self.color_preference]
        if preferred_unvisited_neighbors: return rng.choice(preferred_unvisited_neighbors)
        else: return rng.choice(unvisited_neighbors)

    def traverse_graph(self, graph: nx.Graph, rng: random.Random) -> Tuple[List[int], int, int]:
        """ Simulates exploration walk using deterministic preference strategy. """
        steps_taken = 0; self.preferred_nodes_visited = set()
        try: # Record start node attributes safely
            start_node_attrs = graph.nodes[self.start_node]; self.visited_shapes.append(start_node_attrs.get('shape', 'unknown')); self.visited_shininess.append(start_node_attrs.get('shiny', 'unknown'))
            if start_node_attrs.get('color') == self.color_preference: self.preferred_nodes_visited.add(self.start_node)
        except KeyError: self.visited_shapes.append('unknown'); self.visited_shininess.append('unknown')

        while len(self.preferred_nodes_visited) < self.target_preferred_count and steps_taken < self.max_steps:
            next_node = self.find_next_node(graph, rng) # Calls V5 logic
            if next_node is None: break
            self.visited_nodes.add(next_node); self.path.append(next_node); self.current_node = next_node; steps_taken += 1
            try: # Record visited node attributes safely
                node_attrs = graph.nodes[next_node]; self.visited_shapes.append(node_attrs.get('shape', 'unknown')); self.visited_shininess.append(node_attrs.get('shiny', 'unknown'))
                if node_attrs.get('color') == self.color_preference: self.preferred_nodes_visited.add(next_node) # Add to set
            except KeyError: self.visited_shapes.append('unknown'); self.visited_shininess.append('unknown')

        preferred_nodes_found_count = len(self.preferred_nodes_visited)
        return self.path, preferred_nodes_found_count, steps_taken

# ==============================================================================
# Graph Generation 
# ==============================================================================

def generate_structured_colored_graph(num_nodes: int, colors: List[str], edge_probability: float, structure_seed: int, edge_seed: int, noise_level: float = 0.1) -> nx.Graph:
    G = nx.Graph(); num_colors = len(colors);
    if num_colors == 0: raise ValueError("Color list empty.");
    if not 0.0 <= noise_level <= 1.0: raise ValueError("Noise level invalid.")
    nodes_per_region = num_nodes // num_colors; region_rng = random.Random(structure_seed); node_assignments = {}
    nodes_list = list(range(1, num_nodes + 1)); region_rng.shuffle(nodes_list)
    print(f"  Assigning {num_nodes} nodes to {num_colors} color regions (Noise: {noise_level*100:.1f}%)...")
    for i, node_id in enumerate(nodes_list):
        region_index = min(i // nodes_per_region, num_colors - 1); base_color = colors[region_index]; final_color = base_color
        if noise_level > 0 and region_rng.random() < noise_level:
            possible_noise_colors = [c for c in colors if c != base_color]
            if possible_noise_colors: final_color = region_rng.choice(possible_noise_colors)
        node_assignments[node_id] = final_color
    shape_rng = random.Random(structure_seed + 1); shapes = ['circle', 'square', 'triangle']; shininess_options = ['shiny', 'not_shiny']
    for i in range(1, num_nodes + 1):
         shape = shape_rng.choice(shapes); shininess = shape_rng.choice(shininess_options)
         G.add_node(i, color=node_assignments[i], shape=shape, shiny=shininess)
    edge_rng = random.Random(edge_seed); edge_count = 0
    for i in range(1, num_nodes + 1):
        for j in range(i + 1, num_nodes + 1):
            if edge_rng.random() < edge_probability: G.add_edge(i, j); edge_count += 1
    if num_nodes > 0 and not nx.is_connected(G): warnings.warn(f"Generated graph (seed {structure_seed}) not connected.", stacklevel=2)
    return G

# ==============================================================================
# Simulation (Preference-Driven Goal, Calls Agent V5)
# ==============================================================================
# (Function simulate_agents identical)
def simulate_agents(graph: nx.Graph, num_agents: int, num_traversals_per_agent: int, simulation_seed: int, edge_probability: float, target_preferred_count: int, max_steps: int, colors_list: List[str] = ['blue', 'green', 'red', 'yellow']) -> pd.DataFrame:
    agent_data = []; node_id_to_color_name = {node: graph.nodes[node].get('color', 'unknown') for node in graph.nodes()}
    sim_rng = random.Random(simulation_seed); print(f" Starting simulations for Seed {simulation_seed}...")
    total_simulations = num_agents * num_traversals_per_agent; completed_simulations = 0; skipped_simulations = 0
    print(f"  NOTE: Agent goal: Find {target_preferred_count} preferred nodes (max {max_steps} steps)."); print("  NOTE: Agent using DETERMINISTIC PREFERENCE strategy.");
    nodes_list = list(graph.nodes())
    if not nodes_list: print(" Error: Graph has no nodes."); return pd.DataFrame(agent_data)
    for i in range(total_simulations):
        agent_id = i % num_agents; start_node = sim_rng.choice(nodes_list); color_preference = sim_rng.choice(colors_list)
        agent = Agent(start_node, color_preference, target_preferred_count, max_steps)
        try: path, preferred_nodes_found, steps_taken = agent.traverse_graph(graph, sim_rng); # No bias factor
        except Exception as e: print(f" Error during agent traversal call (Start: {start_node}, Pref: {color_preference}): {repr(e)}"); skipped_simulations += 1; continue
        if not path: print(f"  Warning: Traversal returned empty path (Start: {start_node}). Skipping."); skipped_simulations += 1; continue
        mapped_colors = [node_id_to_color_name.get(node, 'unknown') for node in agent.path]; path_len = len(agent.path); preferred_color_path_count = sum(1 for node in agent.path if graph.nodes.get(node, {}).get('color') == agent.color_preference); pref_prop = (preferred_color_path_count / path_len) if path_len > 0 else 0.0
        agent_data.append({'Seed': simulation_seed, 'Agent': agent_id, 'Start Node': start_node, 'Color Preference': color_preference, 'Target Preferred Count': target_preferred_count, 'Max Steps': max_steps, 'Actual Steps Taken': steps_taken, 'Preferred Nodes Found': preferred_nodes_found, 'Path Length': path_len, 'Preferred Color Path Count': preferred_color_path_count, 'Preferred_Color_Proportion': pref_prop, 'Path': agent.path, 'Mapped Colors': mapped_colors, 'Visited Shapes': agent.visited_shapes, 'Visited Shininess': agent.visited_shininess, 'Density': edge_probability }); completed_simulations += 1
    print(f"  Finished simulations for Seed {simulation_seed}. Completed: {completed_simulations}, Skipped: {skipped_simulations}")
    return pd.DataFrame(agent_data)

# ==============================================================================
# Clustering Feature Engineering / Distance Calculation 
# ==============================================================================

def jaccard_distance(set1: Any, set2: Any) -> float:
    try: set1 = set(set1) if isinstance(set1, (list, tuple, np.ndarray)) else set(set1) if set1 is not None else set(); set2 = set(set2) if isinstance(set2, (list, tuple, np.ndarray)) else set(set2) if set2 is not None else set()
    except TypeError: return 1.0
    intersection = len(set1.intersection(set2)); union = len(set1.union(set2));
    if union == 0: return 0.0
    return 1.0 - intersection / union

def extract_boc_features(group: pd.DataFrame) -> Optional[Any]:
     if 'Mapped Colors' not in group.columns: print("Error:'Mapped Colors'"); return None
     corpus = group['Mapped Colors'].apply(lambda colors: ' '.join(map(str, colors)) if isinstance(colors, list) and colors else '')
     try: vectorizer = CountVectorizer(); X = vectorizer.fit_transform(corpus); print(f"  BoC Features extracted with shape: {X.shape}"); return X
     except Exception as e: print(f"  Error extracting BoC features: {e}"); return None

def extract_combined_features(group: pd.DataFrame) -> Optional[Any]:
    print("  Extracting Combined Features (BoC + Numerical)..."); boc_features = extract_boc_features(group);
    if boc_features is None: return None
    num_cols = ['Path Length', 'Preferred_Color_Proportion'] # these cols exist
    if not all(col in group.columns for col in num_cols): print(f"Error: Missing {num_cols}"); return None
    try: numerical_features = group[num_cols].values.astype(float)
    except Exception as e: print(f"Error accessing numerical features: {e}"); return None
    try: scaler = StandardScaler(); scaled_numerical = scaler.fit_transform(numerical_features)
    except Exception as e: print(f"Error scaling numerical features: {e}"); return None
    try: combined_features = sparse_hstack((boc_features.tocsr(), scaled_numerical), format='csr'); print(f"  Combined Features shape: {combined_features.shape}"); return combined_features
    except Exception as e: print(f"Error combining features: {e}"); return None

def calculate_distance_matrix(group: pd.DataFrame, method: str = 'jaccard', **kwargs) -> Tuple[Optional[np.ndarray], Optional[Any], bool, str]:
    num_samples = len(group); metric_used = method; distance_matrix = None; feature_matrix = None; is_valid = False
    if method == 'jaccard':
        metric_used = 'jaccard';
        if 'Path' not in group.columns: print("Error:'Path'"); return None, None, False, metric_used
        group['Path_Set'] = group['Path'].apply(lambda x: set(x) if isinstance(x, (list, tuple)) else set())
        distance_matrix_calc = np.zeros((num_samples, num_samples)); path_sets = group['Path_Set'].tolist()
        for i in range(num_samples):
            for j in range(i + 1, num_samples):
                try: dist = jaccard_distance(path_sets[i], path_sets[j]);
                except Exception as e: print(f"Error Jaccard({i},{j}): {e}"); return None, None, False, metric_used
                if not np.isfinite(dist): raise ValueError(f"Invalid Jaccard dist: {dist}")
                distance_matrix_calc[i, j] = dist; distance_matrix_calc[j, i] = dist
        distance_matrix = distance_matrix_calc; is_valid = True
    elif method == 'boc':
        metric_used = kwargs.get('metric', 'cosine'); feature_matrix = extract_boc_features(group)
        if feature_matrix is not None:
            try: condensed_distances = pdist(feature_matrix.toarray(), metric=metric_used); distance_matrix = squareform(condensed_distances); print(f"  BoC Features + '{metric_used}' distance matrix calculated."); is_valid = True
            except Exception as e: print(f"Warn: Could not create distance matrix from BoC: {e}"); is_valid = True
        else: return None, None, False, 'boc'
    elif method == 'combined':
         metric_used = kwargs.get('metric', 'euclidean'); feature_matrix = extract_combined_features(group)
         if feature_matrix is not None: is_valid = True; distance_matrix = None; print(f"  Combined features extracted. Assoc metric: '{metric_used}'")
         else: is_valid = False
         return distance_matrix, feature_matrix, is_valid, metric_used
    else: print(f"Error: Unknown method '{method}'."); return None, None, False, 'unknown'
    return distance_matrix, feature_matrix, is_valid, metric_used


# ==============================================================================
# Clustering and Evaluation Function (Unchanged, includes fixes)
# ==============================================================================
def cluster_and_evaluate(df: pd.DataFrame, n_clusters: int = 4, feature_distance_method: str = 'jaccard', cluster_method: str = 'ward', dist_metric_param: str = 'cosine', linkage_method: str = 'ward' ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    
    required_cols = ['Seed', 'Color Preference', 'Mapped Colors'] 
    if feature_distance_method == 'jaccard': required_cols.append('Path')
    elif feature_distance_method == 'combined': required_cols.extend(['Path Length', 'Preferred Color Path Count', 'Path', 'Mapped Colors'])
    elif feature_distance_method == 'boc': required_cols.extend(['Path', 'Mapped Colors'])

    if not all(col in df.columns for col in required_cols): raise ValueError(f"Missing columns for eval: {required_cols}. Available: {df.columns.tolist()}")
    print(f"\n--- Starting Clustering Evaluation ---"); print(f" Feature/Distance Method: {feature_distance_method}"); print(f" Clustering Algorithm: {cluster_method}"); print(f" Target Clusters (k): {n_clusters}")
    results_summary = []; processed_groups = []
    for seed, group in df.groupby('Seed'):
        print(f" Processing Seed {seed}..."); group = group.copy(); num_samples = len(group); group['Cluster'] = -1
        if num_samples < 2: print(f"  Skipping Seed {seed}: Samples < 2."); processed_groups.append(group); continue
        distance_matrix, feature_matrix, is_valid, eval_metric = calculate_distance_matrix(group, method=feature_distance_method, metric=dist_metric_param)
        if not is_valid: processed_groups.append(group); continue
        clusters = None; actual_n_clusters = min(n_clusters, num_samples) if num_samples > 0 else 1;
        if actual_n_clusters < 1: actual_n_clusters = 1
        try: # Clustering Block
            if cluster_method in ['ward', 'complete', 'average', 'single']: # Hierarchical
                 linkage_input_dm = distance_matrix
                 if linkage_input_dm is None:
                      if feature_matrix is not None:
                           print(f"  Calculating '{dist_metric_param}' distance matrix for hierarchical on features...")
                           try: linkage_input_dm = squareform(pdist(feature_matrix.toarray(), metric=dist_metric_param))
                           except Exception as e_pdist: raise ValueError(f"Failed distance calc for hierarchical: {e_pdist}") from e_pdist
                      else: raise ValueError("Neither distance matrix nor features available.")
                 if not isinstance(linkage_input_dm, np.ndarray) or linkage_input_dm.ndim!=2 or linkage_input_dm.shape[0]!=linkage_input_dm.shape[1]: raise ValueError("Invalid DM shape.")
                 if np.isnan(linkage_input_dm).any() or np.isinf(linkage_input_dm).any(): raise ValueError("NaN/Inf in DM.")
                 if not np.allclose(linkage_input_dm, linkage_input_dm.T): warnings.warn(f"DM not symmetric (Seed {seed}).", stacklevel=2); linkage_input_dm = (linkage_input_dm + linkage_input_dm.T) / 2
                 np.fill_diagonal(linkage_input_dm, 0)
                 try: condensed_distance = squareform(linkage_input_dm, checks=True)
                 except ValueError as sq_err: raise ValueError(f"Squareform check failed: {sq_err}") from sq_err
                 if condensed_distance is None or not np.isfinite(condensed_distance).all(): raise ValueError("Invalid condensed distance array.")
                 Z = linkage(condensed_distance, method=cluster_method);
                 if Z is None or not isinstance(Z, np.ndarray): raise TypeError("Linkage failed.")
                 clusters = fcluster(Z, t=actual_n_clusters, criterion='maxclust')
                 if clusters is None or not isinstance(clusters, np.ndarray): raise TypeError("fcluster failed.")
            elif cluster_method in ['kmeans', 'gmm']: # Feature-based
                current_feature_matrix = feature_matrix
                if current_feature_matrix is None:
                     if feature_distance_method == 'boc': _, current_feature_matrix, _, _ = calculate_distance_matrix(group, 'boc', metric=dist_metric_param)
                     elif feature_distance_method == 'combined': _, current_feature_matrix, _, _ = calculate_distance_matrix(group, 'combined', metric=dist_metric_param)
                     if current_feature_matrix is None: raise ValueError(f"Features required for {cluster_method}.")
                if cluster_method == 'kmeans': kmeans = KMeans(n_clusters=actual_n_clusters, random_state=seed, n_init=10, verbose=0); clusters = kmeans.fit_predict(current_feature_matrix); clusters += 1
                elif cluster_method == 'gmm': gmm = GaussianMixture(n_components=actual_n_clusters, random_state=seed, verbose=0, n_init=5); clusters = gmm.fit_predict(current_feature_matrix.toarray()); clusters += 1
            else: raise ValueError(f"Unsupported cluster_method: {cluster_method}")
            group['Cluster'] = clusters; num_unique_clusters_found = len(set(c for c in clusters if c != -1))
        except Exception as e: print(f"  Error during clustering for Seed {seed}: {repr(e)}"); processed_groups.append(group); continue
        # --- Evaluation ---
        silhouette_avg = np.nan; ari_score = np.nan; nmi_score = np.nan
        valid_cluster_mask = group['Cluster'] != -1; clusters_for_eval = group.loc[valid_cluster_mask, 'Cluster']
        num_valid_samples_for_eval = len(clusters_for_eval); num_clusters_for_eval = len(set(clusters_for_eval))
        if num_clusters_for_eval > 1 and num_clusters_for_eval < num_valid_samples_for_eval: # Silhouette Calc
            silhouette_input_data = None; silhouette_metric = 'euclidean' # Default
            can_use_precomputed = (distance_matrix is not None)
            if can_use_precomputed and (feature_distance_method == 'jaccard' or cluster_method in ['ward', 'complete', 'average', 'single']):
                silhouette_metric = 'precomputed'; valid_mask_np = valid_cluster_mask.to_numpy()
                if len(valid_mask_np) == distance_matrix.shape[0]:
                     valid_distance_matrix = distance_matrix[np.ix_(valid_mask_np, valid_mask_np)]
                     if valid_distance_matrix.shape[0] > 1:
                         try: silhouette_avg = silhouette_score(valid_distance_matrix, clusters_for_eval, metric='precomputed')
                         except Exception as e_sil: print(f"   Warning: Silhouette (precomputed) error: {repr(e_sil)}")
                else: print(f"   Warning: Index mismatch Silhouette submatrix (Seed {seed}).")
            elif feature_matrix is not None: # Try features
                current_feature_matrix = feature_matrix
                if current_feature_matrix.shape[0] == num_samples:
                    valid_feature_matrix = current_feature_matrix[valid_cluster_mask]
                    if valid_feature_matrix.shape[0] > 1:
                        try: silhouette_avg = silhouette_score(valid_feature_matrix, clusters_for_eval, metric=eval_metric) # Use metric assoc. with features
                        except Exception as e_sil: print(f"   Warning: Silhouette (feature, metric='{eval_metric}') error: {repr(e_sil)}")
                else: print(f"   Warning: Feature matrix shape mismatch Silhouette (Seed {seed}).")
            if silhouette_avg is np.nan and num_clusters_for_eval > 1:
                 print(f"   Skipping Silhouette: Input data invalid or unavailable (Seed {seed})")
        if num_valid_samples_for_eval > 0: # ARI/NMI Calc
             try: true_labels_for_eval = group.loc[valid_cluster_mask, 'Color Preference']; ari_score = adjusted_rand_score(true_labels_for_eval, clusters_for_eval); nmi_score = normalized_mutual_info_score(true_labels_for_eval, clusters_for_eval)
             except Exception as e_gnd: print(f"   Warning: Could not compute ARI/NMI Seed {seed}: {repr(e_gnd)}")
        formatted_cluster_color_percentages = {}; max_color_separation = np.nan
        try: # Purity Calc
            valid_group_for_purity = group[group['Cluster'] != -1]
            if not valid_group_for_purity.empty:
                if 'Mapped Colors' in valid_group_for_purity.columns:
                    exploded_group = valid_group_for_purity.explode('Mapped Colors').dropna(subset=['Mapped Colors']); exploded_group.rename(columns={'Mapped Colors': 'Flat Colors'}, inplace=True)
                    color_counts = exploded_group.groupby(['Cluster', 'Flat Colors']).size().unstack(fill_value=0)
                    all_possible_colors = sorted(df['Color Preference'].dropna().unique())
                    for color in all_possible_colors:
                        if color not in color_counts.columns: color_counts[color] = 0
                    color_counts = color_counts[all_possible_colors]; cluster_sums = color_counts.sum(axis=1); safe_sums = cluster_sums.replace(0, 1)
                    color_percentages = color_counts.div(safe_sums, axis=0).mul(100); cluster_color_percentages_dict = color_percentages.round(2).apply(lambda r: r.dropna().to_dict(), axis=1).to_dict()
                    formatted_cluster_color_percentages = {int(k): {c: f"{p:.2f}%" for c, p in v.items()} for k, v in cluster_color_percentages_dict.items()}
                    if not color_percentages.empty: max_color_separation = color_percentages.apply(lambda r: r.max() - r.min() if not r.empty else 0.0, axis=1).max()
                    else: max_color_separation = 0.0
                else: print(f"   Warning: 'Mapped Colors' column missing for purity calc (Seed {seed}).")
        except Exception as e: print(f"   Warning: Color percentage error Seed {seed}: {e}")
        results_summary.append({'Seed': seed, 'Algorithm': cluster_method, 'Feature_Metric': f"{feature_distance_method}({eval_metric})", 'Num_Clusters_Target': n_clusters, 'Num_Clusters_Found': num_clusters_for_eval, 'Cluster_Color_Percentage': formatted_cluster_color_percentages, 'Silhouette': silhouette_avg, 'ARI': ari_score, 'NMI': nmi_score, 'Max_Color_Separation': max_color_separation })
        processed_groups.append(group)
    if not processed_groups: warnings.warn("No groups processed."); return pd.DataFrame(results_summary), df.copy()
    df_with_clusters = pd.concat(processed_groups).reset_index(drop=True); results_summary_df = pd.DataFrame(results_summary)
    print("\nClustering and evaluation complete.")
    return results_summary_df, df_with_clusters

# ==============================================================================
# Plotting Functions (Unchanged, includes SyntaxError Fix)
# ==============================================================================

def plot_colored_graph(G: nx.Graph, seed: int, output_dir: str = "plots"):
    if not os.path.exists(output_dir): os.makedirs(output_dir); filepath = os.path.join(output_dir, f"graph_seed_{seed}.png")
    plt.figure(figsize=(12, 10)); pos = nx.spring_layout(G, seed=42); node_colors = [G.nodes[node].get('color', 'gray') for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=200, alpha=0.9); nx.draw_networkx_edges(G, pos, alpha=0.3)
    plt.title(f"Graph Structure (Seed {seed})", fontsize=16); plt.axis('off'); plt.tight_layout(); plt.savefig(filepath); print(f" Saved graph plot to {filepath}"); plt.close()

def plot_color_confusion_matrix(df: pd.DataFrame, cluster_col: str = 'Cluster', true_color_col: str = 'Color Preference', figsize: Tuple[int, int] = (8, 7), cmap: str = 'Blues', output_dir: str = "plots"):
    # (No return value needed - just plots and saves)
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    plot_filename = f"confusion_matrix_{cluster_col}_vs_{true_color_col}.png"; filepath = os.path.join(output_dir, plot_filename)
    # --- Input Validation 
    if cluster_col not in df.columns: raise ValueError(f"Cluster column '{cluster_col}' not found.")
    if true_color_col not in df.columns: raise ValueError(f"True color column '{true_color_col}' not found.")
    # --- End of Fix ---
    df_valid = df[pd.to_numeric(df[cluster_col], errors='coerce').notna()].copy(); df_valid[cluster_col] = df_valid[cluster_col].astype(int); df_valid = df_valid[df_valid[cluster_col] > 0]
    if df_valid.empty: print("Warning: No valid data for confusion matrix."); return
    unique_colors = sorted(df_valid[true_color_col].dropna().unique());
    if not unique_colors: print(f"Error: No valid unique values in '{true_color_col}'."); return
    color_to_int_mapping = {color: i + 1 for i, color in enumerate(unique_colors)}; mapped_color_ints = list(color_to_int_mapping.values()); mapped_color_names = list(color_to_int_mapping.keys())
    print(f"\n--- Confusion Matrix Generation ---"); print(f" Using Color Mapping for Plot Axes: {color_to_int_mapping}")
    try: y_true = df_valid[true_color_col].map(color_to_int_mapping); y_pred = df_valid[cluster_col]
    except Exception as e: print(f" Error preparing y_true/y_pred: {e}"); return
    valid_indices = y_true.notna()
    if not valid_indices.all(): y_true = y_true[valid_indices].astype(int); y_pred = y_pred[valid_indices]
    if len(y_true) == 0: print(" Error: No samples for confusion matrix."); return
    matrix_labels_expected = mapped_color_ints; conf_matrix = confusion_matrix(y_true, y_pred, labels=matrix_labels_expected); conf_matrix_df = pd.DataFrame(conf_matrix, index=mapped_color_names, columns=mapped_color_names)
    plt.figure(figsize=figsize); sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap=cmap, cbar=True, linewidths=.5, linecolor='lightgray', annot_kws={"size": 10})
    plt.title('Confusion Matrix: Predicted Color (by Cluster) vs True Color', fontsize=14); plt.ylabel(f'True Color (`{true_color_col}`)', fontsize=12); plt.xlabel(f'Predicted Color (based on `{cluster_col}` Number)', fontsize=12)
    plt.xticks(rotation=45, ha='right', fontsize=10); plt.yticks(rotation=0, fontsize=10)
    plt.tight_layout(); plt.savefig(filepath); print(f" Saved confusion matrix plot to {filepath}"); plt.close()
    # Removed return statement


# ==============================================================================
# Main Execution Block -> STRUCTURED GRAPH (NO NOISE) + EXPLORE AGENT (V5) + BOC/KMEANS
# Includes Cluster Label Remapping for Plotting
# ==============================================================================
if __name__ == "__main__":

    # --- Parameters ---
    NUM_NODES = 180; COLORS = ['blue', 'green', 'red', 'yellow','white', 'Blue']; EDGE_PROBABILITY = 0.20
    NUM_SEEDS = 40; NUM_AGENTS = 40; NUM_TRAVERSALS_PER_AGENT = 50; # -> 250 records per seed

    # --- Simulation Parameters ---
    TARGET_PREFERRED_COUNT = 5 # Agent goal
    MAX_STEPS = 50             # Step limit
    GRAPH_NOISE_LEVEL = 0.0 # 

    # --- Clustering Set to BOC + KMEANS ---
    N_CLUSTERS = 6
    FEATURE_DISTANCE_METHOD = 'boc'
    CLUSTER_METHOD = 'kmeans'
    DIST_METRIC = 'euclidean' # Metric for Silhouette when using BoC+KMeans features

    # --- Output Directory ---
    OUTPUT_DIR = f"clustering_results_{FEATURE_DISTANCE_METHOD}_{CLUSTER_METHOD}_agent_v5_explore_structured_NO_NOISE" # Updated name
    if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)

    # --- Run Simulation (Agent V5 - Deterministic Preference, NO NOISE Structured Graph) ---
    print(f"--- Starting Simulation Phase (Agent V5 - Deterministic Preference, Structured Graph NO NOISE) ---")
    all_results_list = []; start_time_sim = time.time()
    for seed in range(NUM_SEEDS):
        graph_attribute_seed = seed; graph_edge_seed = seed + NUM_SEEDS
        print(f"\nGenerating structured graph for Seed {seed}...")
        G = generate_structured_colored_graph(NUM_NODES, COLORS, EDGE_PROBABILITY,
                                              graph_attribute_seed, graph_edge_seed,
                                              noise_level=GRAPH_NOISE_LEVEL) # Pass 0.0
        # plot_colored_graph(G, seed, output_dir=os.path.join(OUTPUT_DIR, "graphs")) # Optional
        print(f"Running Simulation for Seed {seed}...")
        simulation_seed = seed
        df_agents = simulate_agents(G, NUM_AGENTS, NUM_TRAVERSALS_PER_AGENT,
                                    simulation_seed, EDGE_PROBABILITY,
                                    target_preferred_count=TARGET_PREFERRED_COUNT,
                                    max_steps=MAX_STEPS,
                                    colors_list=COLORS)
        all_results_list.append(df_agents)
    all_simulation_data = pd.concat(all_results_list, ignore_index=True); end_time_sim = time.time()
    print(f"\n--- Simulation Phase Complete ---"); print(f"Generated {len(all_simulation_data)} records in {end_time_sim - start_time_sim:.2f}s.")

    # --- Run Clustering and Evaluation ---
    print(f"\n--- Starting Clustering Phase ---"); start_time_cluster = time.time()
    summary_results, data_with_clusters = cluster_and_evaluate(
        all_simulation_data.copy(),
        n_clusters=N_CLUSTERS,
        feature_distance_method=FEATURE_DISTANCE_METHOD, # 'boc'
        cluster_method=CLUSTER_METHOD,                 # 'kmeans'
        dist_metric_param=DIST_METRIC                  # 'euclidean'
    )
    end_time_cluster = time.time(); print(f"\n--- Clustering Phase Complete ---"); print(f"Completed in {end_time_cluster - start_time_cluster:.2f}s.")

    # --- Display and Save Results ---
    print("\n--- Clustering Summary Results ---"); pd.set_option('display.max_rows', 50); pd.set_option('display.max_columns', None); pd.set_option('display.width', 120); pd.set_option('display.max_colwidth', 150)
    if not summary_results.empty:
        print(summary_results.round(4).to_string()); avg_ari = summary_results['ARI'].mean(); avg_nmi = summary_results['NMI'].mean(); avg_silhouette = summary_results['Silhouette'].mean()
        print("-" * 50); print(f"Avg ARI: {avg_ari:.4f}"); print(f"Avg NMI: {avg_nmi:.4f}"); print(f"Avg Silhouette: {avg_silhouette:.4f}"); print("-" * 50)
        summary_filename = f"summary_{FEATURE_DISTANCE_METHOD}_{CLUSTER_METHOD}_k{N_CLUSTERS}_agent_v5_explore_structured_NO_NOISE.csv"
        summary_filepath = os.path.join(OUTPUT_DIR, summary_filename)
        try: summary_results.to_csv(summary_filepath, index=False); print(f"Summary results saved to {summary_filepath}")
        except Exception as e: print(f"Error saving summary: {e}")
    else: print("No summary results generated.")


    # --- Generate and Save Confusion Matrix (with Remapping) ---
    if not data_with_clusters.empty and 'Cluster' in data_with_clusters.columns and data_with_clusters['Cluster'].max() > 0 :
         print("\n--- Remapping cluster labels for visualization ---")
         # <<< START REMAPPING LOGIC >>>
         try:
             # Ensure we only use valid clusters for mapping calculation
             valid_clusters_df = data_with_clusters[data_with_clusters['Cluster'] > 0].copy()

             # Find the majority true label for each original cluster number
             # Handle potential multiple modes by taking the first one 
             mode_map = valid_clusters_df.groupby('Cluster')['Color Preference'].agg(
                 lambda x: x.mode()[0] if not x.empty and not x.mode().empty else None
             ).to_dict()

             # Find the desired integer mapping used by the plotting function (based on sorted unique true labels)
             unique_colors = sorted(data_with_clusters['Color Preference'].dropna().unique())
             true_color_to_plot_int = {color: i + 1 for i, color in enumerate(unique_colors)}

             # Create the final remapping: original_cluster_num -> plot_int_representing_majority_color
             remapping_dict = {}
             assigned_plot_ints = set() # To check for conflicts
             print(f"  Majority Color per Original Cluster: {mode_map}")

             for original_cluster_num, majority_color_label in mode_map.items():
                 if majority_color_label is not None:
                     target_int = true_color_to_plot_int.get(majority_color_label)
                     if target_int is not None:
                         if target_int in assigned_plot_ints:
                              warnings.warn(f"Conflict: Cluster {original_cluster_num} also maps to '{majority_color_label}' (Int {target_int}), which was already assigned. Remapped matrix might not be perfectly diagonal.", stacklevel=2)
                         remapping_dict[original_cluster_num] = target_int
                         assigned_plot_ints.add(target_int)
                     else:
                         print(f"  Warning: Could not find target integer for majority color '{majority_color_label}' of cluster {original_cluster_num}")
                         remapping_dict[original_cluster_num] = -1 # Assign error label
                 else:
                      print(f"  Warning: Cluster {original_cluster_num} was empty or had no mode.")
                      remapping_dict[original_cluster_num] = -1 # Assign error label

             print(f"  Cluster Remapping Dict (Original Cluster -> Target Plot Int): {remapping_dict}")

             # Apply the mapping to create a new column
             data_with_clusters['Cluster_Remapped'] = data_with_clusters['Cluster'].map(remapping_dict).fillna(-1).astype(int)

             # Plot using the remapped labels
             print("\n--- Generating Confusion Matrix (using REMAPPED cluster labels) ---")
             plot_output_dir = os.path.join(OUTPUT_DIR, "plots_remapped") # Use a different subdir
             # Call plotting function using the NEW column name
             plot_color_confusion_matrix(
                 data_with_clusters,
                 cluster_col='Cluster_Remapped', # <<< USE REMAPPED COLUMN
                 true_color_col='Color Preference',
                 output_dir=plot_output_dir
             )
             # Rename the saved plot file to indicate it's remapped
             original_plot_name = f"confusion_matrix_Cluster_Remapped_vs_Color Preference.png"
             new_plot_name = f"confusion_matrix_REORDERED_{FEATURE_DISTANCE_METHOD}_{CLUSTER_METHOD}_k{N_CLUSTERS}_agent_v5.png"
             try:
                 # Create dir if it doesn't exist
                 if not os.path.exists(plot_output_dir): os.makedirs(plot_output_dir)
                 os.rename(os.path.join(plot_output_dir, original_plot_name), os.path.join(plot_output_dir, new_plot_name))
                 print(f" Renamed plot to {new_plot_name}")
             except Exception as e_mv:
                 print(f" Could not rename remapped plot: {e_mv}")

         except NameError:
             # This occurs if the plot function definition is missing
             print("ERROR: plot_color_confusion_matrix function not defined.")
         except Exception as e:
             print(f"Could not remap labels or generate remapped confusion matrix plot: {e}")
         # <<< END REMAPPING LOGIC >>>

    else: print("\nSkipping confusion matrix plot.")


    print("\nPipeline finished.")

--- Starting Simulation Phase (Agent V5 - Deterministic Preference, Structured Graph NO NOISE) ---

Generating structured graph for Seed 0...
  Assigning 180 nodes to 6 color regions (Noise: 0.0%)...
Running Simulation for Seed 0...
 Starting simulations for Seed 0...
  NOTE: Agent goal: Find 5 preferred nodes (max 50 steps).
  NOTE: Agent using DETERMINISTIC PREFERENCE strategy.
  Finished simulations for Seed 0. Completed: 2000, Skipped: 0

Generating structured graph for Seed 1...
  Assigning 180 nodes to 6 color regions (Noise: 0.0%)...
Running Simulation for Seed 1...
 Starting simulations for Seed 1...
  NOTE: Agent goal: Find 5 preferred nodes (max 50 steps).
  NOTE: Agent using DETERMINISTIC PREFERENCE strategy.
  Finished simulations for Seed 1. Completed: 2000, Skipped: 0

Generating structured graph for Seed 2...
  Assigning 180 nodes to 6 color regions (Noise: 0.0%)...
Running Simulation for Seed 2...
 Starting simulations for Seed 2...
  NOTE: Agent goal: Find 5 preferred 

AttributeError: 'Series' object has no attribute 'nonzero'

  Majority Color per Original Cluster: {1: 'blue', 2: 'blue', 3: 'white', 4: 'green', 5: 'red', 6: 'blue'}
  Cluster Remapping Dict (Original Cluster -> Target Plot Int): {1: 2, 2: 2, 3: 5, 4: 3, 5: 4, 6: 2}

--- Generating Confusion Matrix (using REMAPPED cluster labels) ---

--- Confusion Matrix Generation ---
 Using Color Mapping for Plot Axes: {'Blue': 1, 'blue': 2, 'green': 3, 'red': 4, 'white': 5, 'yellow': 6}


C:\Users\taodz\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3378: UserWarning: Conflict: Cluster 2 also maps to 'blue' (Int 2), which was already assigned. Remapped matrix might not be perfectly diagonal.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\taodz\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3378: UserWarning: Conflict: Cluster 6 also maps to 'blue' (Int 2), which was already assigned. Remapped matrix might not be perfectly diagonal.
  exec(code_obj, self.user_global_ns, self.user_ns)


 Saved confusion matrix plot to clustering_results_boc_kmeans_agent_v5_explore_structured_NO_NOISE\plots_remapped\confusion_matrix_Cluster_Remapped_vs_Color Preference.png
 Could not rename remapped plot: [WinError 183] Cannot create a file when that file already exists: 'clustering_results_boc_kmeans_agent_v5_explore_structured_NO_NOISE\\plots_remapped\\confusion_matrix_Cluster_Remapped_vs_Color Preference.png' -> 'clustering_results_boc_kmeans_agent_v5_explore_structured_NO_NOISE\\plots_remapped\\confusion_matrix_REORDERED_boc_kmeans_k6_agent_v5.png'

Pipeline finished.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder # If cluster labels are not numeric

# 1. Prepare Data
valid_data = data_with_clusters[data_with_clusters['Cluster'] > 0].copy()

# Target Variable
y = valid_data['Cluster']

# 2. Prepare Features (Using BoC)
corpus = valid_data['Mapped Colors'].apply(lambda colors: ' '.join(map(str, colors if isinstance(colors, list) else [])))
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names_out()

# 3. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 4. Choose and Train Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced') 
print("Training classifier...")
classifier.fit(X_train, y_train)
print(f"Classifier Accuracy on Test Set: {classifier.score(X_test, y_test):.4f}") # Good practice check

# 5. Extract and Analyze Feature Importances
importances = classifier.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("\n--- Top Predictors (Features) for Clusters ---")
print(feature_importance_df.head(10)) # Display top 10 overall predictors
